In [2]:
import numpy as np
# random seed for reproducibility
np.random.seed(123)

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D, MaxPooling2D
#Now we will import some utilities
from keras.utils import to_categorical
#Fixed dimension ordering issue
from keras import backend as K
K.set_image_data_format('channels_first')  # or 'channels_last'

2023-12-07 15:16:58.660884: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 15:16:58.660972: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 15:16:58.689815: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 15:16:58.747656: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 15:16:59.465783: W tensorflow/compiler/tf2

In [4]:
from keras.datasets import mnist

In [5]:
(X_train,y_train),(X_test, y_test)=mnist.load_data()

In [6]:
#Preprocess imput data for Keras
# Reshape input data.
# reshape to be [samples][channels][width][height]
X_train=X_train.reshape(X_train.shape[0],1,28,28)
X_test=X_test.reshape(X_test.shape[0],1,28,28)
# to convert our data type to float32 and normalize our database
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
print(X_train.shape)

(60000, 1, 28, 28)


***Normalization and preprocessing of Data***

In [7]:
# Z-scoring or Gaussian Normalization
X_train=X_train - np.mean(X_train) / X_train.std()
X_test=X_test - np.mean(X_test) / X_test.std()
#(60000, 1, 28, 28)
# convert 1-dim class arrays to 10 dim class metrices
#one hot encoding outputs
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
num_classes=y_test.shape[1]
print(num_classes)
#10
#Define a simple CNN model
print(X_train.shape)
#(60000,1,28,28)

10
(60000, 1, 28, 28)


***Define model architecture***

In [8]:
# create a model
model=Sequential()
model.add(Conv2D(32, (5,5), input_shape=(1,28,28), activation='relu', padding="SAME"))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))  # Set data_format to 'channels_first'
model.add(Dropout(0.3))      # Dropout, one form of regularization
model.add(Flatten())
model.add(Dense(10,activation='elu'))
model.add(Dense(num_classes, activation='softmax'))
print(model.output_shape)

(None, 10)


2023-12-07 15:17:09.248452: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 15:17:09.352285: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 15:17:09.352509: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

***Compile The model***

In [9]:
model.compile(loss='binary_crossentropy', optimizer='adagrad')

In [10]:
X_train.shape

(60000, 1, 28, 28)

***Fit the Model***

In [11]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=200,verbose=1)

2023-12-07 15:17:31.165705: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-07 15:17:32.334108: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f1aa73bd360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-07 15:17:32.334137: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2023-12-07 15:17:32.341520: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1701958652.396840   94133 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


300/300 [==============================] - 3s 3ms/step - loss: 1.0132 - val_loss: 0.8822


***Evaluate Model On the test Data***

In [12]:
# Final evaluation of the model
scores =model.evaluate(X_test, y_test, verbose=0)
print(" CNN Error: %.2f%%" % (scores))
# CNN Error: 17.98%

 CNN Error: 0.88%
